# Features
This notebook generates features and labels (goal/no goal) for all shots and stores them in a HDF file. This is a good practice to save computational time if you want to experiment with multiple pipelines.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import itertools

In [2]:
%load_ext autoreload
%autoreload 2
from soccer_xg.api import DataApi
import soccer_xg.xg as xg
import soccer_xg.features as fs

## Config

In [3]:
# dataset
dir_data = "../data"
provider = 'wyscout_opensource'
leagues = ['ENG', 'ESP', 'ITA', 'GER', 'FRA']
seasons = ['1718']

# features
store_features = f'../data/{provider}/features.h5'

By default, all features defined in `soccer_xg.features.all_features` are computed. It is also possible to compute a subset of these features or add additional feature generators. Each feature generator is a function that expects either a DataFrame object containing actions (i.e., individual actions) or a list of DataFrame objects containing consecutive actions (i.e., game states), and returns the corresponding feature for the individual action or game state. Features that contain information about the shot's outcome are automatically removed.

In [4]:
feature_generators = fs.all_features

## Compute features and labels

In [5]:
for (l,s) in itertools.product(leagues, seasons):
    print(l, s)
    api = DataApi([f"{dir_data}/{provider}/spadl-{provider}-{l}-{s}.h5"])
    xg.get_features(api, xfns=feature_generators).to_hdf(store_features, key=f'{l}/{s}/features', format='table')  
    xg.get_labels(api).to_hdf(store_features, key=f'{l}/{s}/labels', format='table')  

ENG 1718


Generating labels: 100%|██████████| 380/380 [00:08<00:00, 43.51it/s]


ESP 1718


Generating labels: 100%|██████████| 380/380 [00:08<00:00, 43.70it/s]


ITA 1718


Generating labels: 100%|██████████| 380/380 [00:08<00:00, 43.62it/s]


GER 1718


Generating labels: 100%|██████████| 306/306 [00:06<00:00, 43.93it/s]


FRA 1718


Generating labels: 100%|██████████| 380/380 [00:08<00:00, 43.76it/s]


## Load features

In [6]:
features = []
labels = []
for (l,s) in itertools.product(leagues, seasons):
    features.append(pd.read_hdf(store_features, key=f'{l}/{s}/features'))
    labels.append(pd.read_hdf(store_features, key=f'{l}/{s}/labels'))
features = pd.concat(features)
labels = pd.concat(labels)

display(features.head())
display(labels.to_frame().head())

type_id_a0      type_id_a1 type_id_a2 bodypart_id_a0  \
game_id action_id                                                        
2500098 17              shot         dribble      cross           foot   
        40              shot  corner_crossed       pass           foot   
        77              shot       clearance      cross           foot   
        140             shot           cross    dribble           foot   
        145             shot            pass       pass           foot   

                  bodypart_id_a1 bodypart_id_a2 result_id_a1 result_id_a2  \
game_id action_id                                                           
2500098 17                  foot           foot      success      success   
        40                  foot           foot      success         fail   
        77                  foot           foot         fail         fail   
        140                 foot           foot      success      success   
        145                 foot           foot      success      success   

                   start_x_a0  start_y_a0  start_x_a1  start_y_a1  start_x_a2  \
game_id action_id                                                               
2500098 17              99.75       26.52       91.35       29.92       97.65   
        40              91.35       35.36      105.00        0.00       96.60   
        77              75.60       29.92       94.50       27.20       98.70   
        140             92.40       43.52       98.70       51.68       91.35   
        145             99.75       37.40       96.60       38.76       93.45   

                   start_y_a2  end_x_a1  end_y_a1  end_x_a2  end_y_a2  dx_a1  \
game_id action_id                                                              
2500098 17               6.12     99.75     26.52     91.35     29.92   8.40   
        40              23.80     91.35     35.36      0.00     53.72 -13.65   
        77              65.96     75.60     29.92     94.50     27.20 -18.90   
        140             54.40     92.40     43.52     98.70     51.68  -6.30   
        145             45.56     99.75     37.40     96.60     38.76   3.15   

                   dy_a1  movement_a1  dx_a2  dy_a2  movement_a2  dx_a01  \
game_id action_id                                                          
2500098 17         -3.40     9.062009  -6.30  23.80    24.619708     0.0   
        40         35.36    37.903194 -96.60  29.92   101.127476     0.0   
        77          2.72    19.094722  -4.20 -38.76    38.986890     0.0   
        140        -8.16    10.309006   7.35  -2.72     7.837149     0.0   
        145        -1.36     3.431049   3.15  -6.80     7.494164     0.0   

                   dy_a01  mov_a01  dx_a02  dy_a02    mov_a02  \
game_id action_id                                               
2500098 17            0.0      0.0   -8.40    3.40   9.062009   
        40            0.0      0.0  -91.35   18.36  93.176779   
        77            0.0      0.0   18.90   -2.72  19.094722   
        140           0.0      0.0    6.30    8.16  10.309006   
        145           0.0      0.0   -3.15    1.36   3.431049   

                   start_dist_to_goal_a0  start_angle_to_goal_a0  \
game_id action_id                                                  
2500098 17                      9.138539                0.958815   
        40                     13.717584                0.099306   
        77                     29.681752                0.137895   
        140                    15.792099                0.647047   
        145                     6.254798                0.574700   

                   start_dist_to_goal_a1  start_angle_to_goal_a1  \
game_id action_id                                                  
2500098 17                     14.246715                0.290448   
        40                     34.000000                1.570796   
        77                     12.509596                0.574700   
        140             

goal
game_id action_id       
2500098 17         False
        40         False
        77         False
        140        False
        145        False